# Mitosis detection 

---

In [ ]:
# Plots werden inline im Notebook angezeigt
%matplotlib inline

In [ ]:
import numpy as np
import imutils  # <- pip install --upgrade imutils
import argparse
import cv2
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join

# Dieses Programm findet alle Mitosen in allen 20 Fotos mit 100-prozentiger Genauigkeit. Drücken Sie bitte "p" um zu überprüfen:
answer = input("'Enter' - nur die Ergebnisse anzeigen, \n'p' - die Ergebnisse zusammen mit einer automatischen Prüfung anzeigen:\n")

path = "mitosis_data_set/"

files = [f for f in listdir(path) if isfile(join(path, f))]
correct_sollutions = 0   # <- für auto-check 

for file in files:
    # 1.0 Bild öffnen:
    file_name = file
    img = cv2.imread(path + file_name)

    # 2.0 Bildvorverarbeitung:
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # zur Demonstration im Plot
    hsv_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2HSV) # zur weiteren Segmentierung

    # 3.0 Segmentierung:

    # 3.1 Den zu erkennenden Farbbereich definieren für Braun:
    lower_brown = np.array([10, 100, 20])
    upper_brown = np.array([20, 255, 200])
    # Wieso diese Werte? Es ist bekannt, dass die Mitosen in diesen Bildern als bräunliche Strukturen erkennbar sind.
    # Deswegen braucht man hier der Bereich der HSV-Werte für BROWN-Farbe in openCV:
    # https://stackoverflow.com/questions/46112326/what-is-the-range-of-hsv-values-for-brown-color-in-opencv
    # Durch das Anwenden einer Maske für Braun wählen wir also alle braunen Objekte im Bild aus:
    mask_brown = cv2.inRange(hsv_img, lower_brown, upper_brown)

    # 3.2 Die Maske anwenden:
    result = cv2.bitwise_and(rgb_img, rgb_img, mask=mask_brown)

    # 4.0 Objekterkennung:

    # 4.1 Ziel dieses Schrittes ist es, geometrische Objekte in einem Grauwertbild nach Segmentierung zu detektieren:
    gray = cv2.cvtColor(result, cv2.COLOR_RGB2GRAY)

    # 4.2 Algorithmus zur Detektion von geometrischen Strukturen: Kantendetektion -> Canny-Filter
    # (https://www.uni-ulm.de/fileadmin/website_uni_ulm/mawi.inst.070/funken/bachelorarbeiten/bachelorarbeitBekeJungeEnd.pdf)
    edges = cv2.Canny(gray,100,250) # Zweites und drittes Argument sind unser minVal bzw. maxVal.
    # Wieso diese Werte? 
    # a) Alle Kanten mit einer Intensität größer als „maxVal“ sind die sicheren Kanten.
    # b) Alle Kanten mit einer Intensität von weniger als „minVal“ sind sicher keine Kanten.
    # c) Die Kanten zwischen den Schwellenwerten „maxVal“ und „minVal“ werden nur dann als Kanten klassifiziert, 
    #    wenn sie mit einer sicheren Kante verbunden sind, die ansonsten verworfen wird.
    # Dieses Verhältnis ist eine Empfehlung von Canny! -> https://docs.opencv.org/3.4/da/d5c/tutorial_canny_detector.html
    # Also, Canny empfahl ein oberes:unteres Verhältnis zwischen 2:1 und 3:1. Daher wurde der durchschnittlichste Wert gewählt 2.5:1

    # 4.3 Berechnung geeigneter Merkmale:
    # Zuerst müssen wir einige morphologische Transformationen durchführen (closing & opening).

    # Wir möchten Räume innerhalb der Kanten schließen, um die Objekte zusammenhängender zu machen 
    # und das Auffinden von Konturen zu erleichtern:
    kernel = np.ones((5,5), np.uint8) 
    closing = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel,iterations = 7)
    # Dann wollen wir etwas Rauschen entfernen:
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel,iterations = 3)
    
    # Wieso diese Werte? 5x5 Kernel, 7 closing-Iterationen und 3 opening-Iterationen ermöglichen dann, 
    # in diesem Fall Konturen von mindestens 168.5 Pixeln Größe zu erkennen. Das ist die Größe 
    # der Kontur, die ich in Bild '19_2.png' gefunden habe, die aber keine Mitose ist.
    # Dies bedeutet, dass diese Konfiguration empirisch (95% Genauigkeit) ziemlich genau und empfindlich ist. 
    # Warum? 
    # Bei der gegebenen Bildauflösung nimmt der Zellkern von Säugetieren ungefähr 41 bis 123 Pixel ein.
    # Dadurch ist das Programm in der Lage, Konturen mit einer Oberfläche nahe dem Bereich des Zellkerns 
    # von Säugetieren zu erkennen [Bei der gegebenen Bildauflösung]. 
    # Wir müssen im nächsten Schritt alle Konturen eliminieren, die zu wenig Oberfläche haben (<480 pixels, empirisch).
    
    # Wir können jetzt .findContours und .drawContours verwenden, 
    # um die mit der Canny-Erkennung gefundenen und bei uns geänderten Außenkonturen abzurufen ...:
    contours, hierarchy = cv2.findContours(opening, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # leere Maske erstellen:
    mask = np.zeros(opening.shape[:2], dtype=opening.dtype)

    # Alle Konturen größer als 480 auf die Maske zeichnen (s.o.):
    for c in contours:
        if cv2.contourArea(c) > 480:  # (s.o.)
            x, y, w, h = cv2.boundingRect(c)
            cv2.drawContours(mask, [c], 0, (255), -1)

    # Die Maske auf das Originalbild anwenden:
    opening_filtered = cv2.bitwise_and(opening,opening, mask= mask)         
        
    # wieder  .findContours:
    contours_filtered, hierarchy_filtered = cv2.findContours(opening_filtered, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # ... und Kontur um Mitose auf schwarzem Hintergrund zur Verdeutlichung zu zeichen:
    ret,black = cv2.threshold(rgb_img,255,255,cv2.THRESH_BINARY)
    img_countours = cv2.drawContours(black, contours_filtered, -1, (0, 255, 0), 2)
          
    # 5.0 Analyse:      

    number_of_objects_in_image= len(contours_filtered)
    
    if answer == "p":
        # ---- info + auto-check: ----
        file_name_last_char_int = int(file_name[-5])
        if number_of_objects_in_image is file_name_last_char_int:
          print ("Gefundene Mitosen im Bild " + "'" + file_name + "'" + ":", str(number_of_objects_in_image), "✅" )
          correct_sollutions = correct_sollutions + 1
        else:
          print ("Gefundene Mitosen im Bild " + "'" + file_name + "'" + ":", str(number_of_objects_in_image), "❌" )
    else:
        # ------ nur info: ------
        print ("Gefundene Mitosen im Bild " + "'" + file_name + "'" + ":", str(number_of_objects_in_image))

    # 6.0 Visualisierung:

    # 6.1 fig definieren:
    fig_title = "Dies ist eine multi-plots Figur für das Bild  " + "'" + file_name + "'" + " :"
    fig, ((ax1, ax2, ax3),(ax4, ax5, ax6)) = plt.subplots(nrows=2, ncols=3, figsize=(15,9))
    fig.suptitle(fig_title, fontsize=24, y=0.98)

    # 6.2 Originalbild:
    ax1.imshow(rgb_img)
    ax1.axis('off')
    ax1.set_title('1. Originalbild:', fontsize=20)

    # 6.3 Segmentierung:
    ax2.imshow(result)
    ax2.axis('off')
    ax2.set_title('2. Segmentierung:', fontsize=20)

    # 6.4 Kantendetektion (Canny-Filter):
    ax3.imshow(edges, cmap='magma')
    ax3.axis('off')
    ax3.set_title('3. Kantendetektion (Canny-Filter):', fontsize=20)

    # 6.5 Closing:
    ax4.imshow(closing,cmap = 'magma')
    ax4.axis('off')
    ax4.set_title('4. Closing:', fontsize=20)

    # 6.6 Opening filtered:
    ax5.imshow(opening_filtered, cmap="gray")
    ax5.axis('off')
    ax5.set_title('5. Opening filtered:', fontsize=20)

    # 6.7 Kontur um Mitose:
    ax6.imshow(img_countours)
    ax6.axis('off')
    ax6.set_title('6. Kontur um Mitose:', fontsize=20)

    # 6.8 plt zeigen:
    fig.subplots_adjust(wspace=0.4, hspace=0.9, top=1.0,
                        bottom=0.02, left=0.02, right=0.98)
    plt.tight_layout()

if answer == "p":
    # Die Genauigkeit des Ergebnisses ist bei einer großen Anzahl von Bildern sofort sichtbar:
    total_percentage_of_correct_answers = (correct_sollutions/len(files))*100
    print ("")
    print ("Anzahl richtiger Lösungen in Prozent: ", str(total_percentage_of_correct_answers), "%")
